# Segmenting and Clustering Neighborhoods in Toronto

import required libraries

In [1]:
import pandas as pd
import numpy as np
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 1. Download Neighbourhood Data

get data from wikipedia https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

Read the first table with the class attribute containing `sortable`.

In [2]:
canada_postal_codes = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', attrs={'class': 'sortable'})[0]
canada_postal_codes.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [3]:
canada_postal_codes.rename(columns={'Postcode': 'PostalCode', 'Neighbourhood': 'Neighborhood'}, inplace=True)
canada_postal_codes = canada_postal_codes[canada_postal_codes['Borough']!='Not assigned']
canada_postal_codes.reset_index(drop=True, inplace=True)
canada_postal_codes.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Group data by `PostalCode`. For each group:

1. join the neighborhood names with comma to make a new Neighborhood column.
2. use the unique name of borough for the new borough column.

Join two columns by `PostalCode` into a new DataFrame.

In [4]:
grouped = canada_postal_codes.groupby('PostalCode')
Neighborhood = grouped.apply(lambda d: ','.join(d['Neighborhood'])).to_frame()
Borough = grouped.apply(lambda d: pd.unique(d['Borough'])[0]).to_frame()
canada_postal_codes_cleaned = Borough.merge(Neighborhood, on='PostalCode')
canada_postal_codes_cleaned.rename(columns={'0_x': 'Borough', '0_y': 'Neighborhood'}, inplace=True)
canada_postal_codes_cleaned.reset_index(inplace=True)
canada_postal_codes_cleaned.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Locate rows with Neighborhood 'Not assigned' and set to the borough name.

In [5]:
canada_postal_codes_cleaned[canada_postal_codes_cleaned['Neighborhood']=='Not assigned']

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Not assigned


In [6]:
canada_postal_codes_cleaned.at[85, 'Neighborhood'] = canada_postal_codes_cleaned.at[85, 'Borough']

In [7]:
canada_postal_codes_cleaned.loc[85]

PostalCode               M7A
Borough         Queen's Park
Neighborhood    Queen's Park
Name: 85, dtype: object

In [8]:
canada_postal_codes_cleaned.shape

(103, 3)

### Define a function to get latitude and longitude data from given postal code

In [9]:
from geopy.geocoders import Nominatim # geocoder.google deny the request, using geopy
geolocator = Nominatim(user_agent="ny_explorer")

def getLatLng(postal_code):
    location = geolocator.geocode('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = location and [location.latitude, location.longitude] or [None, None]
    return lat_lng_coords

In [10]:
latlng = canada_postal_codes_cleaned.apply(lambda x: pd.Series(getLatLng(x.at['PostalCode']), index=['Latitude', 'Longitude']), axis=1)
latlng

,Latitude,Longitude
0,43.653963,-79.387207
1,43.653963,-79.387207
2,NaN,NaN
3,43.644903,-79.381836
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN


As we cannot get the location data from api, just merge the data prepared.

In [11]:
lat_lng_df = pd.read_csv('http://cocl.us/Geospatial_data')
lat_lng_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
lat_lng_df.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
lat_lng_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
toronto_df = canada_postal_codes_cleaned.merge(lat_lng_df, on='PostalCode')
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## 2. Prepare venue data for neighborhoods in Toronto.

In [14]:
toronto_df = toronto_df[toronto_df['Borough'].str.match(r'.*Toronto.*')]
toronto_df.reset_index(drop=True, inplace=True)

#### Define Foursquare Credentials and Version

check the shape of Toronto neighborhood data

In [15]:
toronto_df.shape

(38, 5)

In [16]:
CLIENT_ID = 'K11CKZSMUKX4N1K4JXVGGCJYO4PUPRGJFRCS0YUGXU3FXR3G' # your Foursquare ID
CLIENT_SECRET = 'GZAMXHIFE4WV2LD2TSW4CBD4WFGQRXTLHITMV3UAI1ONH4WO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: K11CKZSMUKX4N1K4JXVGGCJYO4PUPRGJFRCS0YUGXU3FXR3G
CLIENT_SECRET:GZAMXHIFE4WV2LD2TSW4CBD4WFGQRXTLHITMV3UAI1ONH4WO


#### Define function to process venues data for each neighborhood

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Get venues data for all toronto neighborhoods

In [18]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

#### Check the data size of the DataFrame

In [19]:
print(toronto_venues.shape)
toronto_venues.head()

(1698, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Guru Raghavendra Ji,43.680187,-79.292337,Astrologer
3,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678730,-79.297478,Grocery Store
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


Check venue count for each neighborhood

In [20]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
"Brockton,Exhibition Place,Parkdale Village",19,19,19,19,19,19
Business Reply Mail Processing Centre 969 Eastern,15,15,15,15,15,15
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",15,15,15,15,15,15
"Cabbagetown,St. James Town",45,45,45,45,45,45
Central Bay Street,87,87,87,87,87,87
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


Check the category count

In [21]:
print('unique category count', len(toronto_venues['Venue Category'].unique()))
toronto_venues['Venue Category'].unique()

unique category count 239


array(['Health Food Store', 'Pub', 'Astrologer', 'Grocery Store',
       'Neighborhood', 'Greek Restaurant', 'Ice Cream Shop',
       'Cosmetics Shop', 'Italian Restaurant', 'Brewery', 'Yoga Studio',
       'Fruit & Vegetable Store', 'Pizza Place', 'Restaurant',
       'Bookstore', 'Juice Bar', 'Dessert Shop', 'Trail',
       'Bubble Tea Shop', 'Furniture / Home Store', 'Indian Restaurant',
       'Spa', 'Diner', 'Coffee Shop', 'Bakery', 'Caribbean Restaurant',
       'Café', 'American Restaurant', 'Liquor Store', 'Fish & Chips Shop',
       'Burger Joint', 'Gym', 'Park', 'Sushi Restaurant', 'Steakhouse',
       'Pet Store', 'Burrito Place', 'Fast Food Restaurant',
       'Movie Theater', 'Sandwich Place', 'Board Shop',
       'Food & Drink Shop', 'Fish Market', 'Chinese Restaurant',
       'Comfort Food Restaurant', 'Middle Eastern Restaurant',
       'Cheese Shop', 'Stationery Store', 'Seafood Restaurant',
       'Coworking Space', 'Gastropub', 'Bar', 'Music Store',
       'Latin Ame

#### one-hot encode venue category

In [22]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# there's a category column called 'Neighborhood', we rename it to another name
toronto_onehot.rename(columns={'Neighborhood': 'Category Neighborhood'}, inplace=True)

# add neighborhood column back to dataframe
toronto_onehot.insert(0, 'Neighborhood', toronto_venues['Neighborhood'])

toronto_onehot.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


check the shape of the new frame

In [23]:
toronto_onehot.shape

(1698, 240)

#### Let's group by neighborhood and calculate the mean of the frequency of occurrence of each category

In [24]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.040000,...,0.00000,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.000000,0.066667,0.066667,0.066667,0.133333,0.2,0.133333,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.022222,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.011494,...,0.00000,0.00,0.011494,0.000000,0.000000,0.011494,0.000000,0.000000,0.00,0.011494
7,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00000,0.00,0.060000,0.000000,0.030000,0.010000,0.000000,0.000000,0.00,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,Church and Wellesley,0.011364,0.011364,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.011364,...,0.00000,0.00,0.000000,0.011364,0.011364,0.000000,0.000000,0.011364,0.00,0.011364


make sure the size is as expected

In [25]:
print(toronto_grouped.shape, toronto_df.shape)

(38, 240) (38, 5)


#### Let's print each neighborhood along with the top 5 most common venues

In [26]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3      Thai Restaurant  0.04
4  American Restaurant  0.04


----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1    Cocktail Bar  0.05
2      Restaurant  0.04
3  Farmers Market  0.04
4            Café  0.04


----Brockton,Exhibition Place,Parkdale Village----
                    venue  freq
0          Breakfast Spot  0.11
1                    Café  0.11
2             Coffee Shop  0.11
3                 Stadium  0.05
4  Furniture / Home Store  0.05


----Business Reply Mail Processing Centre 969 Eastern----
            venue  freq
0      Comic Shop  0.07
1   Auto Workshop  0.07
2            Park  0.07
3      Restaurant  0.07
4  Farmers Market  0.07


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0   Airport Service  0.20
1    Airport Loun

#### Let's put that into a pandas dataframe

define a function to return most common venues for a given row

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

create a new dataframe with most common venues for each neighborhood

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,American Restaurant,Bar,Cosmetics Shop,Hotel,Bakery,Burger Joint
1,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Bakery,Steakhouse,Beer Bar,Italian Restaurant,Cheese Shop,Café,Seafood Restaurant
2,"Brockton,Exhibition Place,Parkdale Village",Café,Breakfast Spot,Coffee Shop,Performing Arts Venue,Restaurant,Caribbean Restaurant,Italian Restaurant,Furniture / Home Store,Burrito Place,Stadium
3,Business Reply Mail Processing Centre 969 Eastern,Pizza Place,Garden,Comic Shop,Restaurant,Butcher,Burrito Place,Brewery,Skate Park,Smoke Shop,Fast Food Restaurant
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Terminal,Airport Lounge,Plane,Sculpture Garden,Boutique,Boat or Ferry,Harbor / Marina,Airport Gate,Airport Food Court


## 3. Cluster Neighborhood

Run k-means to cluster the neighborhood into 5 clusters.

In [29]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2],
      dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [30]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

# merge toronto_grouped with toronto_df to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Health Food Store,Grocery Store,Astrologer,Pub,Category Neighborhood,Convenience Store,Cosmetics Shop,Farmers Market,Falafel Restaurant,Event Space
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Bubble Tea Shop,Juice Bar,Spa,Liquor Store,Bookstore
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,2,Sandwich Place,Coffee Shop,Pet Store,Board Shop,Brewery,Burger Joint,Burrito Place,Pub,Pizza Place,Park
3,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Gastropub,Yoga Studio,Latin American Restaurant,Coworking Space,Bookstore
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Bus Line,Park,Lake,Swim School,Yoga Studio,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


Finally, let's visualize the resulting clusters

In [31]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The geograpical coordinate of Toronto are 43.653963, -79.387207.


## 4. Examine Clusters

### Cluster 1

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Lawrence Park,Bus Line,Park,Lake,Swim School,Yoga Studio,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
8,"Moore Park,Summerhill East",Tennis Court,Playground,Park,Dessert Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
10,Rosedale,Park,Playground,Trail,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
23,"Forest Hill North,Forest Hill West",Jewelry Store,Park,Sushi Restaurant,Bus Line,Trail,Yoga Studio,Donut Shop,Discount Store,Dog Run,Doner Restaurant


> The first cluster of neighborhoods can be categorized with 'Park' and 'Playground'

### Cluster 2

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Roselawn,Garden,Yoga Studio,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


> The second cluster of neighborhoods can be categorized with 'Garden' and 'Yoga Studio'

### Cluster 3

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,Health Food Store,Grocery Store,Astrologer,Pub,Category Neighborhood,Convenience Store,Cosmetics Shop,Farmers Market,Falafel Restaurant,Event Space
1,"The Danforth West,Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Bubble Tea Shop,Juice Bar,Spa,Liquor Store,Bookstore
2,"The Beaches West,India Bazaar",Sandwich Place,Coffee Shop,Pet Store,Board Shop,Brewery,Burger Joint,Burrito Place,Pub,Pizza Place,Park
3,Studio District,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Gastropub,Yoga Studio,Latin American Restaurant,Coworking Space,Bookstore
5,Davisville North,Sandwich Place,Gym,Park,Clothing Store,Hotel,Breakfast Spot,Food & Drink Shop,Dumpling Restaurant,Discount Store,Dog Run
6,North Toronto West,Coffee Shop,Clothing Store,Sporting Goods Shop,Gift Shop,Fast Food Restaurant,Mexican Restaurant,Diner,Dessert Shop,Cosmetics Shop,Park
7,Davisville,Pizza Place,Dessert Shop,Sandwich Place,Sushi Restaurant,Italian Restaurant,Coffee Shop,Pharmacy,Café,Brewery,Japanese Restaurant
9,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",Coffee Shop,Pub,Pizza Place,American Restaurant,Liquor Store,Light Rail Station,Sports Bar,Supermarket,Sushi Restaurant,Bagel Shop
11,"Cabbagetown,St. James Town",Coffee Shop,Restaurant,Bakery,Chinese Restaurant,Pizza Place,Pharmacy,Café,Pub,Italian Restaurant,Caribbean Restaurant
12,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Gym,Gastropub,Fast Food Restaurant,Mediterranean Restaurant,Men's Store


> The third cluster of neighborhoods can be categorized with 'Coffee Shop', 'Café' and 'Piza Place'